In [1]:
import tensorflow as tf
import os

In [2]:
## limit the gpu using
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)


In [3]:
import kagglehub

In [28]:
# Download latest version
path = kagglehub.dataset_download("paultimothymooney/chest-xray-pneumonia")

In [29]:
print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/chest-xray-pneumonia


In [31]:
import os
os.listdir(path)

['chest_xray']

In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import kagglehub  # ensure this works for your kagglehub utility

class DATALOAD:
  def __init__(self, flagging, load_mode = "keras"):
    self.flagging = flagging
    self.load_mode = load_mode
    self.mri = "uraninjo/augmented-alzheimer-mri-dataset-v2"
    self.xray = "paultimothymooney/chest-xray-pneumonia"
    self.data_dir = self.get_data_dir()
    self.input_img_size = 224
    self.batch_size = 10
    self.train_ds, self.val_ds = self.load_data()

  def get_data_dir(self):
    if self.flagging == "mri":
      path = kagglehub.dataset_download(self.mri)
      path =  os.path.join(path, 'data')
    else:
      path = kagglehub.dataset_download(self.xray)
      path= os.path.join(path, 'chest_xray')
    return path

  def data_load_keras(self, dir):
    data = tf.keras.preprocessing.image_dataset_from_directory(
        dir,
        image_size=(self.input_img_size, self.input_img_size),
        batch_size =self.batch_size,
        labels='inferred',
        label_mode='categorical',  # for one-hot encoding; use 'int' if you prefer integers
        shuffle=True,
        seed=42)
    return data

  def data_load_pytorch(self, dir):
     transform = transforms.Compose([
            transforms.Grayscale(num_output_channels=3),  # ensure ResNet compatibility
            transforms.Resize((self.input_img_size, self.input_img_size)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # normalize for all 3 channels
        ])
     data = datasets.ImageFolder(dir, transform=transform)
     loader = DataLoader(data, batch_size=self.batch_size, shuffle=True)
     return loader

  def load_data(self):
    if self.load_mode == "keras":
      train_ds = self.data_load_keras(self.data_dir + '/train')
      val_ds = self.data_load_keras(self.data_dir + '/val')
    else:
      train_ds = self.data_load_pytorch(self.data_dir + '/train')
      val_ds = self.data_load_pytorch(self.data_dir + '/val')
    return train_ds, val_ds

xu = DATALOAD("mri", load_mode= "pytorch")
xu.train_ds, xu.val_ds

In [5]:
xu = DATALOAD("mri")

Found 33984 files belonging to 4 classes.
Found 6400 files belonging to 4 classes.


In [6]:
# xu.train_ds, xu.val_ds

In [7]:
# xu = DATALOAD("xray")

In [8]:
# xu.train_ds, xu.val_ds

In [9]:
# xu.train_ds.class_names